In [4]:
import pandas as pd

# Load the dataset (assuming the CSV file is in the working directory)
df = pd.read_excel('data/Online Retail.xlsx')  # use correct encoding to handle special characters
print(df.shape)  # Print the number of rows and columns
df.head()        # Display the first few rows

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Convert InvoiceDate from string to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Remove rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Remove cancellations (negative quantities or negative prices)
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

# Confirm data is cleaned
print("Rows after cleaning:", df.shape[0])
print("Number of unique customers:", df['CustomerID'].nunique())


Rows after cleaning: 397884
Number of unique customers: 4338


In [6]:
# Compute total revenue for the entire period
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
total_revenue = df['TotalPrice'].sum()
print(f"Total revenue in dataset: £{total_revenue:.2f}")

# Compute the number of transactions (invoices) and items
num_invoices = df['InvoiceNo'].nunique()
num_products = df['StockCode'].nunique()
print("Unique invoices:", num_invoices)
print("Unique products:", num_products)

# Compute customer-level metrics
customer_group = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',   # number of distinct invoices (orders) per customer
    'Quantity': 'sum',        # total items bought per customer
    'TotalPrice': 'sum'       # total money spent per customer
}).rename(columns={'InvoiceNo':'OrderCount', 'Quantity':'TotalItems', 'TotalPrice':'TotalSpent'})

print(customer_group[['OrderCount','TotalSpent']].describe())


Total revenue in dataset: £8911407.90
Unique invoices: 18532
Unique products: 3665
        OrderCount     TotalSpent
count  4338.000000    4338.000000
mean      4.272015    2054.266460
std       7.697998    8989.230441
min       1.000000       3.750000
25%       1.000000     307.415000
50%       2.000000     674.485000
75%       5.000000    1661.740000
max     209.000000  280206.020000


In [7]:
# Define a cutoff date for calibration vs holdout period
cutoff_date = pd.Timestamp('2011-09-01')  # use September 1, 2011 as the split

# Split the data into calibration (train) and holdout (future) periods
calibration_df = df[df['InvoiceDate'] < cutoff_date]
holdout_df = df[df['InvoiceDate'] >= cutoff_date]

# Calculate RFM features using the calibration period data
import numpy as np

current_date = calibration_df['InvoiceDate'].max()  # last date in calibration period
# Group by customer on calibration period
rfm = calibration_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (current_date - x.max()).days,  # Recency: days since last purchase in calibration period
    'InvoiceNo': 'nunique',    # Frequency: number of orders in calibration period
    'TotalPrice': 'sum'        # Monetary: total spent in calibration period
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Add Average Order Value
rfm['AvgOrderValue'] = rfm['Monetary'] / rfm['Frequency']

# Add Customer "Tenure" in calibration period (days between first and last purchase + 1)
customer_first_date = calibration_df.groupby('CustomerID')['InvoiceDate'].min()
rfm['Tenure'] = (current_date - customer_first_date).dt.days + 1

rfm.head()


,Recency,Frequency,Monetary,AvgOrderValue,Tenure
CustomerID,,,,,
12346.0,225,1,77183.60,77183.600000,226
12347.0,29,5,2790.86,558.172000,268
12348.0,148,3,1487.24,495.746667,258
12350.0,210,1,334.40,334.400000,211
12352.0,162,5,1561.81,312.362000,197


In [8]:
# Compute actual LTV in holdout period (e.g., total spending Oct-Dec 2011 for each customer)
actual_ltv = holdout_df.groupby('CustomerID')['TotalPrice'].sum().rename("LTV_Future")
# Customers with no purchases in holdout will not appear in this groupby, so we fill them with 0
rfm = rfm.join(actual_ltv, how='left')
rfm['LTV_Future'] = rfm['LTV_Future'].fillna(0)
print(rfm[['Frequency','Monetary','LTV_Future']].head(10))


            Frequency  Monetary  LTV_Future
CustomerID                                 
12346.0             1  77183.60        0.00
12347.0             5   2790.86     1519.14
12348.0             3   1487.24      310.00
12350.0             1    334.40        0.00
12352.0             5   1561.81      944.23
12353.0             1     89.00        0.00
12354.0             1   1079.40        0.00
12355.0             1    459.40        0.00
12356.0             2   2753.08       58.35
12358.0             1    484.86      683.20


In [9]:
# Define feature matrix X and target y
features = ['Recency', 'Frequency', 'Monetary', 'AvgOrderValue', 'Tenure']
X = rfm[features].values
y = rfm['LTV_Future'].values

# Split into train and test sets (e.g., 80/20 split of customers)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [10]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Coefficients of the linear model
coef = linreg.coef_
intercept = linreg.intercept_
print("Intercept:", intercept)
print("Coefficients:", list(zip(features, coef)))

# Predict on test set
y_pred_lr = linreg.predict(X_test)


Intercept: 749.341676026433
Coefficients: [('Recency', 1.881913016743183), ('Frequency', 48.9762438190595), ('Monetary', 0.558148472356389), ('AvgOrderValue', -0.9377146973564898), ('Tenure', -3.971588836477753)]


In [11]:
from sklearn.ensemble import RandomForestRegressor

# Initialize Random Forest with some parameters
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf.fit(X_train, y_train)

# Feature importance can tell us which features the model found most predictive
importances = rf.feature_importances_
print("Feature importances:", list(zip(features, importances)))

# Predict on test set
y_pred_rf = rf.predict(X_test)


Feature importances: [('Recency', 0.015418926875334213), ('Frequency', 0.027398287317184734), ('Monetary', 0.6734283366985867), ('AvgOrderValue', 0.25776192254975044), ('Tenure', 0.025992526559143882)]


In [12]:
# Example only; not executed here
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)


In [13]:
    # Create R, F, M quartile scores (1 = lowest, 4 = highest)
rfm['R_rank'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])  # for Recency, lower = better, so invert labels
rfm['F_rank'] = pd.qcut(rfm['Frequency'], 4, labels=[1,2,3,4])
rfm['M_rank'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])
rfm['RFM_Score'] = rfm['R_rank'].astype(str) + rfm['F_rank'].astype(str) + rfm['M_rank'].astype(str)


ValueError: Bin edges must be unique: Index([1.0, 1.0, 2.0, 4.0, 127.0], dtype='float64', name='Frequency').
You can drop duplicate edges by setting the 'duplicates' kwarg